In [1]:
import numpy as np
import random
from sklearn.neighbors import kneighbors_graph
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html
from sklearn.datasets import load_iris

In [2]:
random.seed(31)

In [3]:
iris_data = load_iris(return_X_y=True)

In [4]:
X_iris, y_iris = iris_data

In [5]:
print("X_iris", X_iris, "y_iris", y_iris, sep="\n")

X_iris
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9

In [6]:
def get_knn_graph_neighs(X, k=5):
    G = kneighbors_graph(X, n_neighbors=k, mode='distance')
    adjacent_matrix = G.toarray()
    n = len(X)
    list_of_neighbors = [[] for i in range(n)]
    for i in range(n):
        for j in range(n):
            if adjacent_matrix[i][j] > 0:
                list_of_neighbors[i].append((j,adjacent_matrix[i][j]))
    return list_of_neighbors

In [7]:
class Vertex:
    def __init__(self, i: int, parents : list = None):
        self.i = i
        self.parents = parents
        self.set_position(np.random.rand(2))
    
    def set_position(self, coordinates):
        self.x = coordinates[0]
        self.y = coordinates[1]
        
    def get_position(self):
        return [self.x, self.y]
        
    def __lt__(self, other): 
        return self.i < other.i
    
    def __str__(self):
        return "Vertex(i=" + str(self.i) + ", parents=" + str(self.parents) + ")"
    
    def __repr__(self):
        return "Vertex(i=" + str(self.i) + ", parents=" + str(self.parents) + ")"

In [8]:
class Graph:
    def __init__(self, vertices, neighs, level=0):
        self.vertices = vertices
        self.neighs = neighs
        self.level = level
        self.n = len(self.vertices)
        self.next_graph = None
        
    def assign_next_graph(G1):
        self.next_graph = G1

In [9]:
def coarse_graph(G: Graph, km: int = 3) -> Graph:
    new_vertices_assignment = {}
    new_vertices = []
    # Do new vertices groupping
    random_order_vertices = list(G.vertices)
    random.shuffle(random_order_vertices)
    for v1 in random_order_vertices:
        if v1 in new_vertices_assignment:
            continue
        neighs_sorted_by_distance = list(sorted([(d, v2) for (v2, d) in G.neighs[v1]]))
        vis = set()
        parents = [v1]
        for (d, v2) in neighs_sorted_by_distance:
            if v2 in vis:
                continue
            vis.add(v2)
            if len(vis) >= km:
                break
            if v2 in new_vertices_assignment:
                continue
            else:
                parents.append(v2)
        sv = Vertex(i=v1.i, parents=parents)
        for v in parents:
            new_vertices_assignment[v] = sv
        new_vertices.append(sv)
        
    
    # Do new neighs calculation
    new_neighs = {}
    for v1 in G.neighs:
        new_v1 = new_vertices_assignment[v1]
        if new_v1 not in new_neighs:
            new_neighs[new_v1] = []
        for (v2, d) in G.neighs[v1]:
            new_v2 = new_vertices_assignment[v2]
            if new_v1 != new_v2:
                new_neighs[new_v1].append((new_v2, d))
    
    # Create next_level graph
    new_G = Graph(new_vertices, new_neighs)
    return new_G

In [10]:
def transform_to_vertices(knn_graph):
    n = len(knn_graph)
    vertices = [Vertex(i) for i in range(n)]
    neighs = {v: [] for v in vertices}
    for i in range(n):
        for (j, d) in knn_graph[i]:
            neighs[vertices[i]].append((vertices[j],d))
    return vertices, neighs

In [11]:
knn_graph_neighs = get_knn_graph_neighs(X_iris)

In [12]:
vertices, neighs = transform_to_vertices(knn_graph_neighs)

In [13]:
vertices

[Vertex(i=0, parents=None),
 Vertex(i=1, parents=None),
 Vertex(i=2, parents=None),
 Vertex(i=3, parents=None),
 Vertex(i=4, parents=None),
 Vertex(i=5, parents=None),
 Vertex(i=6, parents=None),
 Vertex(i=7, parents=None),
 Vertex(i=8, parents=None),
 Vertex(i=9, parents=None),
 Vertex(i=10, parents=None),
 Vertex(i=11, parents=None),
 Vertex(i=12, parents=None),
 Vertex(i=13, parents=None),
 Vertex(i=14, parents=None),
 Vertex(i=15, parents=None),
 Vertex(i=16, parents=None),
 Vertex(i=17, parents=None),
 Vertex(i=18, parents=None),
 Vertex(i=19, parents=None),
 Vertex(i=20, parents=None),
 Vertex(i=21, parents=None),
 Vertex(i=22, parents=None),
 Vertex(i=23, parents=None),
 Vertex(i=24, parents=None),
 Vertex(i=25, parents=None),
 Vertex(i=26, parents=None),
 Vertex(i=27, parents=None),
 Vertex(i=28, parents=None),
 Vertex(i=29, parents=None),
 Vertex(i=30, parents=None),
 Vertex(i=31, parents=None),
 Vertex(i=32, parents=None),
 Vertex(i=33, parents=None),
 Vertex(i=34, parents=No

In [14]:
neighs

{Vertex(i=0, parents=None): [(Vertex(i=4, parents=None), 0.1414213562373093),
  (Vertex(i=17, parents=None), 0.09999999999999998),
  (Vertex(i=27, parents=None), 0.14142135623730995),
  (Vertex(i=28, parents=None), 0.14142135623730995),
  (Vertex(i=39, parents=None), 0.14142135623730964)],
 Vertex(i=1, parents=None): [(Vertex(i=9, parents=None), 0.17320508075688784),
  (Vertex(i=12, parents=None), 0.1414213562373099),
  (Vertex(i=25, parents=None), 0.22360679774997896),
  (Vertex(i=34, parents=None), 0.14142135623730964),
  (Vertex(i=45, parents=None), 0.14142135623730986)],
 Vertex(i=2, parents=None): [(Vertex(i=3, parents=None), 0.24494897427831802),
  (Vertex(i=6, parents=None), 0.264575131106459),
  (Vertex(i=12, parents=None), 0.264575131106459),
  (Vertex(i=45, parents=None), 0.264575131106459),
  (Vertex(i=47, parents=None), 0.14142135623730978)],
 Vertex(i=3, parents=None): [(Vertex(i=2, parents=None), 0.24494897427831802),
  (Vertex(i=29, parents=None), 0.17320508075688812),
 

In [15]:
G0 = Graph(vertices, neighs)

In [16]:
coarse_graph(G0)

In [17]:
Gs = [G0]
while True:
    G_l = Gs[-1]
    G_l1 = coarse_graph(G_l)
    Gs.append(G_l1)
    G_l.next_graph = G_l1
    
    if len(G_l1.vertices) > 0.8 * len(G_l.vertices):
        break

In [18]:
Gs

In [27]:
import random

from sklearn.neighbors import DistanceMetric
import numpy as np
from sklearn.neighbors import kneighbors_graph

dist = DistanceMetric.get_metric('euclidean')

N = 10
M = 5
gamma = 7


def probability_i_j(X, i, j, sigma, suma):
    return count_exp_dist(X, i, j, sigma) / suma


def count_exp_dist(X, i, j, sigma):
    return np.exp(-dist(X[i], X[j]) ** 2 / (2 * sigma ** 2))


def count_6(Y_i, Y_j):
    return 1 + dist(Y_i, Y_j) ** 2


def count_p_i_cond_el(X, sigma, i, el, are_neighbours):
    if not are_neighbours:
        return 0
    suma = 0
    for k in range(len(X)):
        if k != i:
            suma += count_exp_dist(X, i, k, sigma)
    return probability_i_j(X, i, el, sigma, suma)


def count_p_i_el(X, sigma, i, el, are_neighbours, N):
    p_i_cond_el = count_p_i_cond_el(X, sigma, i, el, are_neighbours)
    p_el_cond_i = count_p_i_cond_el(X, sigma, el, i, are_neighbours)
    return p_i_cond_el + p_el_cond_i / 2 * N


def compute_gradient(X, vertexes, adj_matrix):
    # (11)
    original_vertexes = np.copy(vertexes)
    n = len(vertexes)
    i = random.randint(0, n-1)
    y_i = vertexes[i]
    el = random.randint(0, n-1)
    while el == i:
        el = random.randint(0, n-1)
    y_el = vertexes[el]  # positive sample
    rest_of_ys = np.delete(vertexes, [i, el], None)  # negative samples
    sigma = 0.1
    are_neighbours = el in adj_matrix[i]
    p_i_L = count_p_i_el(X, sigma, i, el, are_neighbours, N)
    suma = - 2 * p_i_L * (y_i - y_el) / count_6(y_i, y_el)
    for k in rest_of_ys:
        if k in adj_matrix[i]:
            y_el_k = rest_of_ys[k]
#             are_neighbours = el in adj_matrix[k]
#             count_p_i_el(X, sigma, k, el, are_neighbours, N)
            suma += 2 * gamma *  p_i_L * (y_i - y_el_k) / ((dist(y_i, y_el_k)) ** 2 * count_6(y_i, y_el_k))

    return suma


In [30]:
def iterative_refinement(G):
    for i, Gi in enumerate(reversed(G)):
        if i == 0:
            continue
#             rands = np.random.rand(len(vertices), 2)
#             for vertex, rand in zip(vertices, rands):
#                 vertex.set_position(rand)
        else:
            vertices_coordinates = [vertex.get_position() for vertex in  Gi.vertices]
            neighs = pass
        # TODO [ for vertex in Gi.next_graph.neighs]
#             print(Gi.next_graph.neighs)
            compute_gradient(G0.vertices, vertices_coordinates, neighs)
            print(Gi)
        

In [31]:
iterative_refinement(Gs)

TypeError: 'Vertex' object is not iterable